In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, roc_curve, auc

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('student-mat.csv', sep=';')
#df = pd.read_csv('student-por.csv', sep=';')
print("Shape:",df.shape)
df = df.drop(columns=['G1','G2'])
df = df.rename(columns={'G3': 'target'})
df.head()

In [ ]:
cat_cols = df.select_dtypes(include='object').columns.tolist()
df[cat_cols] = df[cat_cols].apply(LabelEncoder().fit_transform)
df.head()

In [ ]:
X = df.drop(columns=['target'])
y = tf.keras.utils.to_categorical(df['target'], num_classes=21)

In [ ]:
overfit = Sequential([
    Dense(128, activation='relu', input_shape=(30,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(21, activation='softmax')
])

overfit.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)
history1 = overfit.fit(x_train, y_train, epochs=25,
                    batch_size=8, validation_split=0.2,
                    verbose=1)

In [ ]:
regularized = Sequential([
    Dense(128, activation='relu', input_shape=(30,), kernel_regularizer=regularizers.l2(0.002)),
    Dropout(0.25),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.002)),
    Dropout(0.25),
    Dense(21, activation='softmax')
])

regularized.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                  patience=10, 
                                                  restore_best_weights=True)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
history2 = regularized.fit(x_train, y_train, epochs=25, 
                    batch_size=16, validation_split=0.3,
                    callbacks=[early_stopping], verbose=1)

In [ ]:
def plot_history(history1, history2):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history1.history['accuracy'], label='Overfit Model Validation Accuracy')
    plt.plot(history2.history['accuracy'], label='Regularized Model Validation Accuracy')
    plt.title('Training Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history1.history['val_accuracy'], label='Overfit Model Validation Accuracy')
    plt.plot(history2.history['val_accuracy'], label='Regularized Model Validation Accuracy')
    plt.title('Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_history(history1, history2)

In [ ]:
loss1, accuracy1 = overfit.evaluate(x_train, y_train)
loss2, accuracy2 = overfit.evaluate(x_test, y_test)
print(f'Train accuracy Overfit Model: {accuracy1:.4f}')
print(f'Test accuracy Overfit Model: {accuracy2:.4f}')

In [ ]:
loss1, accuracy1 = regularized.evaluate(x_train, y_train)
loss2, accuracy2 = regularized.evaluate(x_test, y_test)
print(f'Train accuracy Regularized Model: {accuracy1:.4f}')
print(f'Test accuracy Regularized Model: {accuracy2:.4f}')

In [ ]:
y_pred1 = np.argmax(overfit.predict(x_test), axis=1)
y_test1 = np.argmax(y_test, axis=1)
y_pred2 = np.argmax(regularized.predict(x_test), axis=1)
y_test2 = np.argmax(y_test, axis=1)

In [ ]:
print("Classification Report for Overfit:")
print(classification_report(y_test1, y_pred1))

print("\nClassification Report for Regularized:")
print(classification_report(y_test2, y_pred2))

In [ ]:
labels1 = np.unique(np.concatenate((y_test1, y_pred1)))
labels2 = np.unique(np.concatenate((y_test2, y_pred2)))

cm1 = confusion_matrix(y_test1, y_pred1, labels=labels1)
cm2 = confusion_matrix(y_test2, y_pred2, labels=labels2)

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

ConfusionMatrixDisplay(cm1, display_labels=labels1).plot(ax=ax[0], colorbar=False)
ax[0].set_title("Overfit Confusion Matrix", size=25)

ConfusionMatrixDisplay(cm2, display_labels=labels2).plot(ax=ax[1], colorbar=False)
ax[1].set_title("Regularized Confusion Matrix", size=25)

plt.tight_layout()
plt.show()